# 인과적 언어 모델링을 위한 프롬프트 튜닝

프롬프트는 특정 작업에 맞는 입력 텍스트를 추가하여 언어 모델의 동작을 안내하는 데 도움이 됩니다. 프롬프트 튜닝은 사전 훈련된 모델에 새로 추가된 프롬프트 토큰만 훈련하고 업데이트하는 추가적인 방법입니다. 이렇게 하면 가중치가 고정된 사전 훈련된 모델 하나를 사용하고, 각 다운스트림 작업에 대해 별도의 모델을 완전히 미세 조정하는 대신 더 작은 프롬프트 매개변수 집합을 훈련하고 업데이트할 수 있습니다. 모델이 점점 더 커짐에 따라 프롬프트 튜닝은 더 효율적일 수 있으며 모델 매개변수가 확장됨에 따라 결과도 훨씬 좋아집니다.

<Tip>

💡 프롬프트 튜닝에 대해 자세히 알아보려면 [매개변수 효율적인 프롬프트 튜닝을 위한 스케일의 힘](https://arxiv.org/abs/2104.08691)을 읽어보세요.

</Tip>

이 가이드에서는 프롬프트 튜닝을 적용하여 [RAFT](https://huggingface.co/datasets/ought/raft) 데이터 세트의 `twitter_complaints` 하위 집합에서 [`bloomz-560m`](https://huggingface.co/bigscience/bloomz-560m) 모델을 훈련하는 방법을 보여줍니다.

시작하기 전에 필요한 모든 라이브러리가 설치되어 있는지 확인하세요.

```bash
!pip install -q peft transformers datasets
```

## 설정

모델과 토크나이저, 훈련할 데이터 세트 및 데이터 세트 열, 일부 훈련 하이퍼파라미터, [PromptTuningConfig](https://huggingface.co/docs/peft/main/en/package_reference/tuners#peft.PromptTuningConfig)를 정의하는 것으로 시작합니다. [PromptTuningConfig](https://huggingface.co/docs/peft/main/en/package_reference/tuners#peft.PromptTuningConfig)에는 작업 유형, 프롬프트 임베딩을 초기화할 텍스트, 가상 토큰 수, 사용할 토크나이저에 대한 정보가 포함됩니다.

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, default_data_collator, get_linear_schedule_with_warmup
from peft import get_peft_config, get_peft_model, PromptTuningInit, PromptTuningConfig, TaskType, PeftType
import torch
from datasets import load_dataset
import os
from torch.utils.data import DataLoader
from tqdm import tqdm

device = "cuda"
model_name_or_path = "bigscience/bloomz-560m"
tokenizer_name_or_path = "bigscience/bloomz-560m"
peft_config = PromptTuningConfig(
    task_type=TaskType.CAUSAL_LM,
    prompt_tuning_init=PromptTuningInit.TEXT,
    num_virtual_tokens=8,
    prompt_tuning_init_text="트윗이 불만 사항인지 아닌지 분류합니다.",
    tokenizer_name_or_path=model_name_or_path,
)

dataset_name = "twitter_complaints"
checkpoint_name = f"{dataset_name}_{model_name_or_path}_{peft_config.peft_type}_{peft_config.task_type}_v1.pt".replace(
    "/", "_"
)
text_column = "Tweet text"
label_column = "text_label"
max_length = 64
lr = 3e-2
num_epochs = 50
batch_size = 8

## 데이터 세트 로드

이 가이드에서는 [RAFT](https://huggingface.co/datasets/ought/raft) 데이터 세트의 `twitter_complaints` 하위 집합을 로드합니다. 이 하위 집합에는 `complaint` 또는 `no complaint`로 레이블이 지정된 트윗이 포함됩니다.

In [ ]:
dataset = load_dataset("ought/raft", dataset_name)
dataset["train"][0]
{"Tweet text": "@HMRCcustomers No this is my first job", "ID": 0, "Label": 2}

`Label` 열을 더 읽기 쉽게 만들려면 `Label` 값을 해당 레이블 텍스트로 바꾸고 `text_label` 열에 저장합니다. [map](https://huggingface.co/docs/datasets/main/en/package_reference/main_classes#datasets.Dataset.map) 함수를 사용하여 이 변경 사항을 전체 데이터 세트에 한 번에 적용할 수 있습니다.

In [ ]:
classes = [k.replace("_", " ") for k in dataset["train"].features["Label"].names]
dataset = dataset.map(
    lambda x: {"text_label": [classes[label] for label in x["Label"]]},
    batched=True,
    num_proc=1,
)
dataset["train"][0]
{"Tweet text": "@HMRCcustomers No this is my first job", "ID": 0, "Label": 2, "text_label": "no complaint"}

## 데이터 세트 전처리

다음으로 토크나이저를 설정하고, 시퀀스 패딩에 사용할 적절한 패딩 토큰을 구성하고, 토큰화된 레이블의 최대 길이를 결정합니다.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id
target_max_length = max([len(tokenizer(class_label)["input_ids"]) for class_label in classes])
print(target_max_length)
3

`preprocess_function`을 만들어 다음을 수행합니다.

1. 입력 텍스트와 레이블을 토큰화합니다.
2. 배치의 각 예제에 대해 토크나이저의 `pad_token_id`로 레이블을 패딩합니다.
3. 입력 텍스트와 레이블을 `model_inputs`로 연결합니다.
4. `labels` 및 `model_inputs`에 대한 별도의 어텐션 마스크를 만듭니다.
5. 배치의 각 예제를 다시 반복하여 입력 ID, 레이블, 어텐션 마스크를 `max_length`로 패딩하고 PyTorch 텐서로 변환합니다.

In [ ]:
def preprocess_function(examples):
    batch_size = len(examples[text_column])
    inputs = [f"{text_column} : {x} Label : " for x in examples[text_column]]
    targets = [str(x) for x in examples[label_column]]
    model_inputs = tokenizer(inputs)
    labels = tokenizer(targets)
    for i in range(batch_size):
        sample_input_ids = model_inputs["input_ids"][i]
        label_input_ids = labels["input_ids"][i] + [tokenizer.pad_token_id]
        # print(i, sample_input_ids, label_input_ids)
        model_inputs["input_ids"][i] = sample_input_ids + label_input_ids
        labels["input_ids"][i] = [-100] * len(sample_input_ids) + label_input_ids
        model_inputs["attention_mask"][i] = [1] * len(model_inputs["input_ids"][i])
    # print(model_inputs)
    for i in range(batch_size):
        sample_input_ids = model_inputs["input_ids"][i]
        label_input_ids = labels["input_ids"][i]
        model_inputs["input_ids"][i] = [tokenizer.pad_token_id] * (
            max_length - len(sample_input_ids)
        ) + sample_input_ids
        model_inputs["attention_mask"][i] = [0] * (max_length - len(sample_input_ids)) + model_inputs[
            "attention_mask"
        ][i]
        labels["input_ids"][i] = [-100] * (max_length - len(sample_input_ids)) + label_input_ids
        model_inputs["input_ids"][i] = torch.tensor(model_inputs["input_ids"][i][:max_length])
        model_inputs["attention_mask"][i] = torch.tensor(model_inputs["attention_mask"][i][:max_length])
        labels["input_ids"][i] = torch.tensor(labels["input_ids"][i][:max_length])
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

[map](https://huggingface.co/docs/datasets/main/en/package_reference/main_classes#datasets.Dataset.map) 함수를 사용하여 `preprocess_function`을 전체 데이터 세트에 적용합니다. 모델에 더 이상 필요하지 않으므로 처리되지 않은 열을 제거할 수 있습니다.

In [ ]:
processed_datasets = dataset.map(
    preprocess_function,
    batched=True,
    num_proc=1,
    remove_columns=dataset["train"].column_names,
    load_from_cache_file=False,
    desc="Running tokenizer on dataset",
)

`train` 및 `eval` 데이터 세트에서 [`DataLoader`](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader)를 만듭니다. 데이터 세트의 샘플이 CPU에 있는 경우 훈련 중 GPU로의 데이터 전송 속도를 높이려면 `pin_memory=True`를 설정합니다.

In [ ]:
train_dataset = processed_datasets["train"]
eval_dataset = processed_datasets["test"]


train_dataloader = DataLoader(
    train_dataset, shuffle=True, collate_fn=default_data_collator, batch_size=batch_size, pin_memory=True
)
eval_dataloader = DataLoader(eval_dataset, collate_fn=default_data_collator, batch_size=batch_size, pin_memory=True)

## 훈련

모델을 설정하고 훈련을 시작할 준비가 거의 다 되었습니다!

[AutoModelForCausalLM](https://huggingface.co/docs/transformers/main/en/model_doc/auto#transformers.AutoModelForCausalLM)에서 기본 모델을 초기화하고 `peft_config`와 함께 `get_peft_model()` 함수에 전달하여 [PeftModel](https://huggingface.co/docs/peft/main/en/package_reference/peft_model#peft.PeftModel)을 만듭니다. 새 [PeftModel](https://huggingface.co/docs/peft/main/en/package_reference/peft_model#peft.PeftModel)의 훈련 가능한 매개변수를 인쇄하여 원래 모델의 전체 매개변수를 훈련하는 것보다 얼마나 효율적인지 확인할 수 있습니다!

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_name_or_path)
model = get_peft_model(model, peft_config)
print(model.print_trainable_parameters())
"trainable params: 8192 || all params: 559222784 || trainable%: 0.0014648902430985358"

옵티마이저와 학습률 스케줄러를 설정합니다.

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=(len(train_dataloader) * num_epochs),
)

모델을 GPU로 옮긴 다음 훈련 루프를 작성하여 훈련을 시작합니다!

In [ ]:
model = model.to(device)

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for step, batch in enumerate(tqdm(train_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        total_loss += loss.detach().float()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

    model.eval()
    eval_loss = 0
    eval_preds = []
    for step, batch in enumerate(tqdm(eval_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)
        loss = outputs.loss
        eval_loss += loss.detach().float()
        eval_preds.extend(
            tokenizer.batch_decode(torch.argmax(outputs.logits, -1).detach().cpu().numpy(), skip_special_tokens=True)
        )

    eval_epoch_loss = eval_loss / len(eval_dataloader)
    eval_ppl = torch.exp(eval_epoch_loss)
    train_epoch_loss = total_loss / len(train_dataloader)
    train_ppl = torch.exp(train_epoch_loss)
    print(f"{epoch=}: {train_ppl=} {train_epoch_loss=} {eval_ppl=} {eval_epoch_loss=}")

## 모델 공유

원하는 경우 Hub에서 모델을 저장하고 공유할 수 있습니다. Hugging Face 계정에 로그인하고 메시지가 표시되면 토큰을 입력합니다.

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

[push_to_hub](https://huggingface.co/docs/transformers/main/en/main_classes/model#transformers.PreTrainedModel.push_to_hub) 함수를 사용하여 Hub의 모델 리포지토리에 모델을 업로드합니다.

In [ ]:
peft_model_id = "your-name/bloomz-560m_PROMPT_TUNING_CAUSAL_LM"
model.push_to_hub("your-name/bloomz-560m_PROMPT_TUNING_CAUSAL_LM", use_auth_token=True)

모델이 업로드되면 모델 파일 크기가 33.5kB에 불과하다는 것을 알 수 있습니다! 🤏

## 추론

추론을 위해 샘플 입력에서 모델을 사용해 보겠습니다. 모델을 업로드한 리포지토리를 보면 `adapter_config.json` 파일이 표시됩니다. 이 파일을 [PeftConfig](https://huggingface.co/docs/peft/main/en/package_reference/config#peft.PeftConfig)에 로드하여 `peft_type` 및 `task_type`을 지정합니다. 그런 다음 프롬프트 튜닝된 모델 가중치와 구성을 [from_pretrained()](https://huggingface.co/docs/peft/main/en/package_reference/peft_model#peft.PeftModel.from_pretrained)에 로드하여 [PeftModel](https://huggingface.co/docs/peft/main/en/package_reference/peft_model#peft.PeftModel)을 만듭니다.

In [ ]:
from peft import PeftModel, PeftConfig

peft_model_id = "stevhliu/bloomz-560m_PROMPT_TUNING_CAUSAL_LM"

config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path)
model = PeftModel.from_pretrained(model, peft_model_id)

트윗을 가져와 토큰화합니다.

In [ ]:
inputs = tokenizer(
    f'{text_column} : {"@nationalgridus 물이 나오지 않고 요금은 현재 지불되었습니다. 이에 대해 조치를 취해 주시겠습니까?"} Label : ',
    return_tensors="pt",
)

모델을 GPU에 놓고 예측된 레이블을 *생성*합니다.

In [ ]:
model.to(device)

with torch.no_grad():
    inputs = {k: v.to(device) for k, v in inputs.items()}
    outputs = model.generate(
        input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], max_new_tokens=10, eos_token_id=3
    )
    print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True))
[
    "Tweet text : @nationalgridus 물이 나오지 않고 요금은 현재 지불되었습니다. 이에 대해 조치를 취해 주시겠습니까? Label : complaint"
]